**NOTE: this step has already been implemented in the "real_commuting_for_ED.csv" in the data folder, do not need to run again.**

To be run before effective distance file, to ensure that all regions meet the SIR model's criteria. The SIR model does not run if the rows in the population-commuter table are too low, so this file calculates the table as done in the EpiCommute script, and removes those below a certain threshold. 

In [1]:
import pandas as pd
import numpy as np
import glob

In [6]:
data_path = '/Users/shivyucel/Documents/SDS_2021.nosync/SDS_2020-2021/SDS_Thesis/Data/'

In [ ]:
pop = pd.read_csv(data_path + 'h3/population/h3_pop.csv')

## Clean up original commuting table 

In [ ]:
df = pd.read_csv(infile)
df = df[['ZONA_O', 'ZONA_D', 'final_count']]
df = df.merge(h3_ids, left_on='ZONA_O', right_on='Unnamed: 0')
df = df.merge(h3_ids, left_on='ZONA_D', right_on='Unnamed: 0')

df = df.rename(columns={'0_x': 'SOURCE', '0_y': 'TARGET', 'final_count': 'FLUX'})[['SOURCE', 'TARGET', 'FLUX']]
df1 = pd.DataFrame(df.groupby(['SOURCE', 'TARGET'])['FLUX'].agg(sum))
df1.reset_index(inplace=True)

df1.to_csv(data_path + 'paper_clean_commute.csv')

### Run population-commuting table, script, filter out regions with a sum below threshold (100), and save as correct_indices.csv

In [7]:
df1 = pd.read_csv(data_path + 'h3/paper_clean_commute.csv') #generated at the end of data_analysis.ipynb
df1 = df1[['SOURCE', 'TARGET', 'FLUX']]

In [8]:
h3ids = pd.read_csv('/Users/shivyucel/Documents/SDS_2021.nosync/SDS_2020-2021/SDS_Thesis/Data/h3/h3_IDs.csv')

h3ids.rename(columns={'Unnamed: 0': 'hex'}, inplace=True)

In [ ]:

pivoted = df1.pivot(index="SOURCE", columns="TARGET", values="FLUX").fillna(0)


mxm_mobility = pivoted.values


relevant = pd.DataFrame(pivoted.index)

relevant = relevant.merge(h3ids, left_on='SOURCE', right_on = '0')

pop = pd.read_csv(data_path + 'h3/population/h3_pop.csv')

merged = pop.merge(relevant, left_on='h3', right_on='hex')

merged = merged.sort_values(by='0')


populations = merged['2020_pop_h3'].values

mobility_subpops = mxm_mobility.sum(axis=1)

# Normalize the mobility matrix for each row (origin)
mobility_normalized = (mxm_mobility.T / mobility_subpops).T 

population = (mobility_normalized.T * populations).T

# Create the population-commuter table
population = np.round(population).astype(int) 

print(population.sum(axis=1).min())



In [25]:
master_lst = []
lst = []
while population.sum(axis=1).min() <= 100:

    if lst == []:
        infile = data_path + 'h3/paper_clean_commute.csv'
        df = pd.read_csv(infile)[['SOURCE', 'TARGET', 'FLUX']]
        df1 = pd.DataFrame(df.groupby(['SOURCE', 'TARGET'])['FLUX'].agg(sum))
        df1.reset_index(inplace=True)
        pivoted = df1.pivot(index="SOURCE", columns="TARGET", values="FLUX").fillna(0)
    else:
        pivoted.drop(index=lst, inplace=True)
        pivoted.drop(labels=lst, axis=1, inplace=True)
    
    lst = []
    mxm_mobility = pivoted.values
    
    h3ids = pd.read_csv(data_path + 'h3/commuting/h3_IDs.csv')
    
    h3ids.rename(columns={'Unnamed: 0': 'hex'}, inplace=True)
    
    
    relevant = pd.DataFrame(pivoted.index)
    
    relevant = relevant.merge(h3ids, left_on='SOURCE', right_on = '0')
    
    pop = pd.read_csv(data_path + 'h3/population/h3_pop.csv')
    
    merged = pop.merge(relevant, left_on='h3', right_on='hex')
    
    merged = merged.sort_values(by='0')
    
    populations = merged['2020_pop_h3'].values
    
    mobility_subpops = mxm_mobility.sum(axis=1)
    
    # Normalize the mobility matrix for each row (origin)
    mobility_normalized = (mxm_mobility.T / mobility_subpops).T 
    
    population = (mobility_normalized.T * populations).T
    
    # Create the population-commuter table
    population = np.round(population).astype(int) 
    
    df3 = pd.DataFrame(population, index=pivoted.index, columns=pivoted.columns)
    
    for ind, row in df3.iterrows():
        if row.sum() <= 100:
            lst.append(ind)
            master_lst.append(ind)
    
    print(population.sum(axis=1).min())

0
105


In [35]:
df1 = df1[~df1['SOURCE'].isin(master_lst)]
df1 = df1[~df1['TARGET'].isin(master_lst)]

In [44]:
pivoted = df1.pivot(index="SOURCE", columns="TARGET", values="FLUX").fillna(0)

In [ ]:
merged = h3_ids.merge(pop, left_on='Unnamed: 0', right_on='h3')

In [ ]:
filtered_pop = merged[merged['0'].isin(pivoted.index)]['2020_pop_h3']

### Create new DataFrames for effective distance calculation (SIR_preprocessed_commuting) and for SIR Model (mxm_updated_real and new_h3_pop) 

In [ ]:
#np.savetxt(data_path + 'h3/mxm_updated_real.csv', pivoted.values,  delimiter =',')

In [67]:
#np.savetxt(data_path + 'h3/population/new_h3_pop.txt', filtered_pop.values)

In [72]:
#pivoted.to_csv('SIR_preprocessed_commuting.csv')